In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import nltk
from nltk.corpus import stopwords

In [ ]:
train = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/train.csv")
test = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/test.csv")
sample = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/sample_submission.csv")

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
# Collecting the info about the train data
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
# We have null value for text and selected_text only
train.dropna(inplace = True)

In [ ]:
train.isnull().sum()

In [ ]:
# Collecting info for the test data
test.info()

**EDA**

In [ ]:
train.head()

Selected_text is subset of text

In [ ]:
train.describe()

lets look at the distribution of tweets in the train set

In [ ]:
temp = train.groupby('sentiment').count()['text'].reset_index().sort_values(by = 'text', ascending = False)
temp.style.background_gradient(cmap = 'Purples')

In [ ]:
# Plotting the sentiments count
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (12,6))
sns.countplot(x = 'sentiment', data = train)

In [ ]:
fig = go.Figure(go.Funnelarea(
text = temp.sentiment,
values = temp.text,
title = {"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}
))
fig.show()


In [ ]:
def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c))/(len(a) + len(b) -len(c))

In [ ]:
results_jaccard = []

for ind, row in train.iterrows():
    sentence1 = row.text
    sentence2 = row.selected_text
    
    jaccard_score = jaccard(sentence1, sentence2)
    results_jaccard.append([sentence1, sentence2,jaccard_score])

In [ ]:
train.iterrows()

In [ ]:
jaccard = pd.DataFrame(results_jaccard, columns = ["text", "selected_text", "jaccard_score"])
train = train.merge(jaccard, how = 'outer')

In [ ]:
jaccard.head()

In [ ]:
train['Num_words_ST'] = train['selected_text'].apply(lambda x: len(str(x).split())) # Number of words in Selected Text
train['Num_word_text'] = train['text'].apply(lambda x: len(str(x).split())) # Number of words in main text
train['difference_in_words'] = train['Num_word_text'] - train['Num_words_ST'] # Difference in number of words text and selected text

In [ ]:
train.head()

Let's look at the distribution of Meta-Features

In [ ]:
hist_data = [train['Num_words_ST'], train['Num_word_text']]

group_labels = ['Selected_Text', 'Text']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, show_curve = False)
fig.update_layout(title_text = 'Distribtion of Number of words')
fig.update_layout(
autosize = False,
width = 900,
height = 700,
paper_bgcolor = 'LightSteelBlue'
)
fig.show()

In [ ]:
hist_data = [train['Num_words_ST'], train['Num_word_text']]

group_labels = ['Selected_Text', 'Text']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, show_curve = True)
fig.update_layout(title_text = 'Distribtion of Number of words')
fig.update_layout(
autosize = False,
width = 900,
height = 700,
paper_bgcolor = 'LightSteelBlue'
)
fig.show()

The number of words plot is really interesting, the tweets having number of words greater than 25 are very less and thus the number of words distribution plot is right skewed.

In [ ]:
plt.figure(figsize = (12,6))
p1 = sns.kdeplot(train['Num_words_ST'], shade = True, color = "r").set_title('Kernel Distribution of Number of words')
p1 = sns.kdeplot(train['Num_word_text'], shade = True, color = 'b')

In [ ]:
plt.figure(figsize = (12,6))
p1 = sns.kdeplot(train[train['sentiment']=='positive']['difference_in_words'], shade = True, color = "b").set_title('Kernel Distribution of Difference in Number of Words')
p2 = sns.kdeplot(train[train['sentiment']=='negative']['difference_in_words'], shade = True, color = 'r')

In [ ]:
plt.figure(figsize = (12,6))
sns.distplot(train[train['sentiment']=='neutral']['difference_in_words'], kde = False)

since most of the value obtained in difference in numbers of word is zero so we can.t plot kde plot for sentiment analysis hence we have taken kde = False in the above case

In [ ]:
k = train[train['sentiment']=='neutral']['difference_in_words']
k

In [ ]:
plt.figure(figsize = (12,6))
p1 = sns.kdeplot(train[train['sentiment']=='positive']['jaccard_score'], shade = True, color = 'b').set_title("KDE of Jaccard Scores across different Sentiments")
p2 = sns.kdeplot(train[train['sentiment']=='negative']['jaccard_score'], shade = True, color = 'r')
plt.legend(labels = ['positive', 'negative'])

In [ ]:
plt.figure(figsize = (12,6))
sns.distplot(train[train['sentiment']=='neutral']['jaccard_score'], kde = False)

In [ ]:
k = train[train['Num_word_text']<=2]

In [ ]:
k.groupby('sentiment').mean()['jaccard_score']

In [ ]:
train['Num_words_ST'].unique()

In [ ]:
len(k)

In [ ]:
len(k[k['sentiment']=='positive'])

In [ ]:
k[k['sentiment']=='positive']

Cleaning the Corpus

In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove links, remove punctuation and remove words containing numbers'''
    text = str(text).lower()
    text = re.sub('\[*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\W*\d\w*', '', text)
    return text

In [ ]:
train['text'] = train['text'].apply(lambda x: clean_text(x))
train['selected_text'] = train['text'].apply(lambda x: clean_text(x))

In [ ]:
train.head()

In [ ]:
train['temp_list'] = train['selected_text'].apply(lambda x: str(x).split())
top = Counter([item for sublist in train['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words', 'count']
temp.style.background_gradient(cmap = 'Blues')

In [ ]:
fig = px.bar(temp, x = 'count', y = 'Common_words', title = 'Common Words Text', orientation = 'h',
            width = 700, height = 700, color = 'Common_words')
fig.show()

  Removing the stop words

In [ ]:
def remove_stopword(x):
    return [y for y in x if y not in stopwords.words('english')]
train['temp_list'] = train['temp_list'].apply(lambda x: remove_stopword(x))

In [ ]:
top = Counter([item for sublist in train['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp = temp.iloc[1:,:]
temp.columns = ['Common_words', 'count']
temp.style.background_gradient(cmap = 'Purples')

In [ ]:
fig = px.treemap(temp, path = ['Common_words'], values = 'count', title = 'Tree of Most Common Words')
fig.show()

**Most Common Words in Text**

In [ ]:
train['temp_list1'] = train['text'].apply(lambda x: str(x).split()) # List of words in every row for text
train['temp_list1'] = train['temp_list1'].apply(lambda x: remove_stopword(x)) # Removing Stopwords

In [ ]:
top = Counter([item for sublist in train['temp_list1'] for item in sublist])
temp = pd.DataFrame(top.most_common(25))
temp = temp.iloc[1:, :]
temp.columns = ['Common_words', 'count']
temp.style.background_gradient(cmap = 'Blues')

In [ ]:
fig = px.bar(temp, x = 'count', y = "Common_words", title = "Common Words in Text",
            orientation = 'h', width = 700, height = 700, color = 'Common_words')
fig.show()

**Most Common Words Sentiments Wise**

In [ ]:
Positive_sent = train[train['sentiment'] == 'positive']
Negative_sent = train[train['sentiment'] == 'negative']
Neutral_sent = train[train['sentiment'] == 'neutral']

In [ ]:
# MOst Common Positive Words
top = Counter([item for sublist in Positive_sent['temp_list'] for item in sublist])
temp_positive = pd.DataFrame(top.most_common(20))
temp_positive.columns = ['Common_words','count']
temp_positive.style.background_gradient(cmap = 'Greens')

In [ ]:
# Most Common Negative Words
top = Counter([item for sublist in Negative_sent['temp_list'] for item in sublist])
temp_negative = pd.DataFrame(top.most_common(20))
temp_negative = temp_negative.iloc[1:,:]
temp_negative.columns = ['Common_words', 'count']
temp_negative.style.background_gradient(cmap = 'Reds')

In [ ]:
fig = px.treemap(temp_negative, path = ['Common_words'], values = 'count', title = 'Tree of Most Common Negative Words')
fig.show()

In [ ]:
# Most Common Neutral words
top = Counter([item for sublist in Neutral_sent['temp_list'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.iloc[1:,:]
temp_neutral.columns = ['Common_words', 'count']
temp_neutral.style.background_gradient(cmap = 'Reds')

In [ ]:
fig = px.bar(temp_neutral, x = 'count', y = 'Common_words', title = 'Most Common Neutral Words',
            orientation = 'h', width = 700, height = 700, color = 'Common_words')
fig.show()

In [ ]:
fig = px.treemap(temp_neutral, path = ['Common_words'], values = 'count', title = 'Tree of Most Common Neutral Words')
fig.show()

**Let's Look at Unique Words in each Segment**

In [ ]:
raw_text = [word for word_list in train['temp_list1'] for word in word_list]

In [ ]:
raw_text

In [ ]:
def words_unique(sentiment, numwords, raw_words):
    '''
    Input:
        segment - Segment Category (ex. 'Neutral');
        numwords - how many specific words do you want to see in the final result;
        raw_word = list for item in train_data[train_data.segments == segments]['temp_list1']:
    
    Output:
    dataframe giving information about the name of the specific ingredient and how many times it
    occurs in the chosen cuisine (in descending order based on their counts)...
    '''
    
    allother = []
    for item in train[train.sentiment != sentiment]['temp_list1']:
        for word in item:
            allother.append(word)
    allother = list(set(allother))
        
    specificnonly = [x for x in raw_text if x not in allother]
    
    mycounter = Counter()
    
    for item in train[train.sentiment == sentiment]['temp_list1']:
        for word in item:
            mycounter[word] += 1
    
    keep = list(specificnonly)
    
    for word in list(mycounter):
        if word not in keep:
            del mycounter[word]
            
    Unique_words = pd.DataFrame(mycounter.most_common(numwords), columns = ['words', 'count'])
    
    return Unique_words

In [ ]:
Unique_Positive= words_unique('positive', 20, raw_text)
print("The top 20 unique words in Positive Tweets are:")
Unique_Positive.style.background_gradient(cmap='Greens')

In [ ]:
fig = px.treemap(Unique_Positive, path = ['words'], values = 'count', title = 'Tree Of Unique Positive Words')
fig.show()

In [ ]:
from palettable.colorbrewer.qualitative import Pastel1_7
plt.figure(figsize = (16,10))
my_circle = plt.Circle((0,0), 0.7, color = 'white')
plt.pie(Unique_Positive['count'], labels = Unique_Positive.words, colors = Pastel1_7.hex_colors)
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.title('DoNut Plot Of Unique Positive Words')
plt.show()

In [ ]:
Unique_Negative = words_unique('negative', 10, raw_text)
print("The top 10 unique words in Negative Tweets are:")
Unique_Negative.style.background_gradient(cmap = 'Reds')

In [ ]:
from palettable.colorbrewer.qualitative import Pastel1_7
plt.figure(figsize = (16,10))
my_circle = plt.Circle((0,0), 0.7, color = 'white')
plt.rcParams['text.color'] = 'black'
plt.pie(Unique_Negative['count'], labels = Unique_Negative.words, colors = Pastel1_7.hex_colors)
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.title('DoNut Plot Of Unique Negative Words')
plt.show()

In [ ]:
Unique_Neutral = words_unique('neutral', 10, raw_text)
print("The top 10 unique words in Neutral Tweets are:")
Unique_Neutral.style.background_gradient(cmap = 'Oranges')

In [ ]:
from palettable.colorbrewer.qualitative import Pastel1_7
plt.figure(figsize = (16,10))
my_circle = plt.Circle((0,0), 0.7, color = 'white')
plt.pie(Unique_Neutral['count'], labels = Unique_Neutral.words, colors = Pastel1_7.hex_colors)
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.title('DoNut Plot Of Unique Neutral Words')
plt.show()

**It's Time For WordClouds**

In [ ]:
def plot_wordcloud(text, mask = None, max_words = 200, max_font_size = 100,)